In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

나이브 베이즈(Naive Bayes)는 서로 조건부 독립(conditional independence)인 피쳐를 가정하고, 베이즈 이론을 기반으로 하는 머신러닝 알고리즘이다. 확률 변수 $Y$가 주어졌을 때 $X_1$과 $X_2$가 조건부 독립이면 아래 식을 만족한다.

$$P(X_1, X_2|Y) = P(X_1|Y)P(X_2|Y)$$

나이브 베이즈 알고리즘을 활용해 와인 종류를 구분한다.

In [3]:
# 데이터 불러오기
from sklearn import datasets # 와인 데이터를 사용하기 위해 import 한다.
raw_wine = datasets.load_wine() # 와인 데이터를 불러온다.

In [4]:
# 피쳐/타겟 데이터 지정
X = raw_wine.data # 와인 피쳐 데이터를 저장한다.
y = raw_wine.target # 와인 타겟 데이터를 저장한다.

In [10]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝/테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0) # 트레이닝 데이터와 테스트 데이터로 분할한다.
print(X_tn.shape, X_te.shape)

(133, 13) (45, 13)


In [11]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn) # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [12]:
# 데이터 학습
from sklearn.naive_bayes import GaussianNB # 나이브 베이즈 알고리즘을 사용하기 위해 import 한다.
clf_gnb = GaussianNB() # 나이브 베이즈 모델 객체를 만든다.
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 선형 회귀 분석 알고리즘을 학습시킨다.
clf_gnb.fit(X_tn_std, y_tn)

GaussianNB()

In [13]:
# 데이터 예측
pred_gnb = clf_gnb.predict(X_te_std) # 표준화된 테스트 데이터 X_te_std로 예측한다.
print(pred_gnb)

[0 2 1 0 1 1 0 2 1 1 2 2 0 0 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [22]:
# 재현율 평가
from sklearn.metrics import recall_score # 재현율 평가하기 위해 import 한다.
# recall_score() 함수의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_gnb)를 넘겨 재현율을 평가한다.
# average 옵션으로 재현율 계산 방법을 지정한다. 기본값은 binary이고 예측 결과가 2가지 일 때 사용한다.
# multi-class 인데 average 옵션을 지정하지 않으면 기본값 binary로 지정되서 에러가 발생된다.
# None: 각 클래스별 재현율을 리턴한다.
# micro: 전체 클래스에 대해 각 클래스에 대한 TP, FP, FN을 합한 뒤에 재현율을 계산한다.
# macro: 각 클래스에 대해 재현율을 계산한 뒤 산술 평균을 계산해서 재현율을 계산한다.
# weighted: 각 클래스에 대해 재현율을 게산한 뒤 각 클래스별 데이터 비율에 따른 가중 평균으로 재현율을 계산한다.
recall = recall_score(y_te, pred_gnb, average='macro')
print(recall)

0.9523809523809524


In [23]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_gnb)
print(conf_matrix)

[[16  0  0]
 [ 2 18  1]
 [ 0  0  8]]


In [24]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.86      0.92        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.94        45
weighted avg       0.94      0.93      0.93        45

